# Abfrage der SRU-Schnittstelle von Alma
Code aus dem Library Carpentry Workshop der UB Basel vom 12.4.2021-13.04.2021

### Vorgehen und Ziel

Das Ziel von diesem Projekt war es, anhand von MMS-ID (Systemnummern) aus Alma die SRU-Schnittstelle der IZ Basel nach den dazugehörigen bibliografischen Daten abzufragen. Zudem soll ausgewählt werden können, welche MARC-Felder ausgegeben werden sollen und alles soll statt im XML-Format in Tabellenform ausgegeben werden.

Was ist SRU? <br>
https://de.wikipedia.org/wiki/Search/Retrieve_via_URL <br>
https://slsp.ch/de/metadata <br>
https://witzigs.gitlab.io/cas-dmit-metadaten/daten_beziehen/sru/requests/ <br>
SRU-Schnittstelle IZ Basel: https://slsp-network.alma.exlibrisgroup.com/view/sru/41SLSP_UBS

In [1]:
#Für das Vorgehen werden einige Bibliotheken benötigt, die in Python importiert werden müssen 

#urllib.request --> braucht es um die "SRU-URLs" gut lesbar abzubilden.
import urllib.request
#pandas as pd --> pandas ist eine Bibliothek, die hilft Daten zu verwalten. Hier wird sie z.B. gebraucht um Excel auszulesen. 
import pandas as pd
#time --> time wird hier benötigt, um die Arbeitsschritte zwischenzeitlich zu pausieren, damit die SRU-Schnittstelle nicht zu viele Anfragen auf einmal bearbeiten muss. 
import time
#bs4 BeautifulSoup --> Beautfulsoup hilft, die MARC-XML in CSV umzuwandeln. 
from bs4 import BeautifulSoup
#csv --> Die csv-Bibliothek hilft, die csv-Datei zu erstellen. 
import csv


#Als ersters wird die benötigte Excel-Datei mit den MMS-IDs ausgelesen. Wichtig ist, dass das Excelfile im gleichen Ordner wie das Jupyter-Notebook oder das py-file gespeichert ist.
df = pd.read_excel ('DOKSF_NOT_DOKSDIZS_NOT_DOKSEZS.xlsx')

#In einem weiteren Arbeitsschritt wird die benötigte Excelspalte ausgelesen. (In Anführungszeichen in der eckigen Klammer.)
#Die IDs werden ausgelesen und mit dem Befehlt "tolist" in einer Liste gespeichert.
id_list = df['MMS-ID'].tolist()

#Mit den IDs kann nun für jede Aufnahme ein Link auf die SRU-Schnittstelle erstellt werden. Der Link ist immer gleich aufgebaut, die MMS-ID soll am Schluss eingefügt werden.
sru_anfang = 'https://slsp-network.alma.exlibrisgroup.com/view/sru/41SLSP_UBS?version=1.2&operation=searchRetrieve&recordSchema=marcxml&query=alma.mms_id=='

#Ein csv "f" wird im Schreibmodus geöffnet. Achtung, dass encoding UTF-8 macht danach allenfalls in Excel Probleme und muss manuell auf UTF-16 gesetzt werden.
f = csv.writer(open("daten_dump_DOKSF_NOT_DOKSDIZS_NOT_DOKSEZS.csv", "w", encoding="UTF-8"))

#Hier werden die gewünschten Bezeichnungen der Spalten definiert. In Anführungszeichen und mit Komma abgetrennt.
f.writerow(["001", "110 a", "110 0", "710 a", "710 0", "520 a", "245 a", "245 p", "856 u", "990 f", "008", "264 c"])



#Hier werden Funktionen erstellt, um die benötigsten Infos aus den MARC-Feldern zu holen.
#Es gibt unglaubliche viele Arten von MARC-Feldern (mit und ohne Unterfelder, wiederholbar nicht wiederholbar etc.)
#Hier wurden für zwei Arten Funktionen erstellt:
#1. Unwiederholbare Felder ohne Unterfeld (z.B. 001 mit MMS-ID)
#2. Felder, die mehrmals vorkommen können. Die Unterfelder sind aber pro Feld einmalig. (z.B. Schlagwörter in 6XX-Felder)
#Mit diesem Script sind also nicht alle Arten von Feldern abgedeckt. Mit einer weiteren Funktion kann dies aber hier ergänzt werden.


def unwiederholbare_ohne_unterfeld(feld_tag):
    #Bei beiden Feldern wurde try- und except eingefügt, weil ansonsten häufig aufgrund eines fehlerhaften Datensatzes das ganze Script abgebrochen wird. Aber daher ist auch empfehlenswert zu prüfen, ob wirklich alle Datensätze in mein CSV übernommen wurden.
    try:
        unwiederholbares_feld = soup.find(tag=feld_tag).get_text()
        return(unwiederholbares_feld)
    except:
        pass
    
def unterfeldurchgehen(feld_tag, unterfeld_code):
    try:
        feldname_list = [unterfeld.find(code=unterfeld_code).get_text() 
                         for unterfeld in soup.find_all(tag=feld_tag)]
        #wenn es mehere MARC-Feldern mit den gleichen Unterfeldern gibt, werden die Unterfelder mit einem | abgetrennt
        #weil es bei der Verarbeitung von csv in Excel immer Probleme gibt, wurden hier ; zu / umgewandelt. Dies ist aber fakultativ.
        beautiful_string = "| ".join(feldname_list).replace(";", "/")
        return beautiful_string
    except:
        pass


#Hier beginnt die eigentliche Programmschleife. Die einzelnen IDs aus der Excelliste werden laufend zum SRU-String zusammengefügt.
#Mit Hilfe der Bibliothek "urllib" werden die einzelnen Links ausgelesen.
for cur_id in id_list:
    sru_strings= sru_anfang + str(cur_id)
    
    sru_data = urllib.request.urlopen(sru_strings).read()
   
 #Damit die SRU-Schnittselle nicht überfodert wird, werden die Abfragen nur im 2-Sekunden-Takt ausgeführt.   
    time.sleep(2)
#Mit Hilfe der Bibliothek "BeautifulSoup" werden die erhaltenen Daten als XML erkannt und können ausgelesen werden.
    soup = BeautifulSoup(sru_data, 'xml')

    
 #Die letzte Zeile kann nach Bedarf angepasst werden. Hier werden nämlich die MARC-Felder definiert, die in der CSV-Datei auglesen werden sollen.
#Dafür werden die oben definierten Funktionen ausgeführt. Wichtig ist, dass die ausgewählten MARC-Felder mit den oben definierten Spaltennamen übereinstimmten.
    f.writerow([ unwiederholbare_ohne_unterfeld("001"), unterfeldurchgehen("110", "a"), unterfeldurchgehen("110", "0"), unterfeldurchgehen("710", "a"), unterfeldurchgehen("710", "0"), unterfeldurchgehen("520", "a"), unterfeldurchgehen("245", "a"),  unterfeldurchgehen("245", "p"), unterfeldurchgehen("856", "u"), unterfeldurchgehen("990", "f"),  unwiederholbare_ohne_unterfeld("008") , unterfeldurchgehen("264", "c") ])



### Alternatives Vorgehen

Das obenstehende Script beschreibt ein Vorgehen, dass mit bereits bekannten Metadaten in einer Excelliste arbeitet.
Natürlich ist es auch möglich, direkt eine SRU-Abfrage zu formuliren und danach die so gewonnen Metadaten genau gleich als CSV auszudrucken.

Beispiel:
Suche nach Noten vor 1600:
https://swisscovery.slsp.ch/view/sru/41SLSP_UBS?version=1.2&operation=searchRetrieve&recordSchema=marcxml&query=main_pub_date<=1600%20and%20alma.content_type_code=ntm


Folgendes ist dabei unbedingt zu beachten: die SRU-Schnittstelle gibt höchstens 10 Ergebnisse zurück. Um weitere Ergebnisse zu erhalten muss der Link mit "&startRecord=11" usw. ergänzt werden.

Dazu gibt es auch sehr spannende Tutorials bei der DNB:
https://www.dnb.de/DE/Professionell/Services/WissenschaftundForschung/DNBLab/dnblab_node.html#doc731014bodyText3
https://hub.gke2.mybinder.org/user/deutsche-nation-bliothek-dnblab-7mydyelo/notebooks/DNB_SRU_Tutorial.ipynb
Man kann die Tutorials der DNB auch gleich als Jupyter Notebook laden, was wirklich super ist!


##### Vielen Dank an das "Team 4" (Iris, Christina, Johanna) für den Input dazu.

In [ ]:
#Für das Vorgehen werden einige Bibliotheken benötigt, die in Python importiert werden müssen 

#urllib.request --> braucht es um die "SRU-URLs" gut lesbar abzubilden.
import urllib.request
#pandas as pd --> pandas ist eine Bibliothek, die hilft Daten zu verwalten. Hier wird sie z.B. gebraucht um Excel auszulesen. 
import pandas as pd
#time --> time wird hier benötigt, um die Arbeitsschritte zwischenzeitlich zu pausieren, damit die SRU-Schnittstelle nicht zu viele Anfragen auf einmal bearbeiten muss. 
import time
#bs4 BeautifulSoup --> Beautfulsoup hilft, die MARC-XML in CSV umzuwandeln. 
from bs4 import BeautifulSoup
#csv --> Die csv-Bibliothek hilft, die csv-Datei zu erstellen. 
import csv


#Die vorher formulierte Abfrage wird als "Basis-Url" gespeichert.
sru_basis = "https://swisscovery.slsp.ch/view/sru/41SLSP_UBS?version=1.2&operation=searchRetrieve&recordSchema=marcxml&query=main_pub_date<=1600%20and%20alma.content_type_code=ntm"

#Die Basis-URL wird geöffneet und ausgelesen.
sru_data = urllib.request.urlopen(sru_basis).read()

#Mit Hilfe der Bibliothek "BeautifulSoup" werden die erhaltenen Daten als XML erkannt und können ausgelesen werden.

soup = BeautifulSoup(sru_data, 'xml')

#Es wird eine Variable erstellt mit der Gesamtanzahl der Records, welche die Abfrage enthaltet.
num_records = int(soup.find("numberOfRecords").text)


#Mit Hilfe einer Vorschleife wird der Zusatz "&startRecord=" erstellt. Es wird für jeden einzelnen Record einen Link mit diesem Zusatz erstellt und in eine Liste gespeichert.

liste_paramater_links = []


for parameter in range(num_records):

     if parameter < num_records:

        liste_paramater_links.append("&startRecord="+str(parameter+1))

#Ein csv "f" wird im Schreibmodus geöffnet. Achtung, dass encoding UTF-8 macht danach allenfalls in Excel Probleme und muss manuell auf UTF-16 gesetzt werden.
f = csv.writer(open("ntm_vor_1600.csv", "w", encoding="UTF-8"))

#Hier werden die gewünschten Bezeichnungen der Spalten definiert. In Anführungszeichen und mit Komma abgetrennt.
f.writerow(["001", "110 a", "110 0", "710 a", "710 0", "520 a", "245 a", "245 p", "856 u", "990 f", "008", "264 c"])



#Hier werden Funktionen erstellt, um die benötigsten Infos aus den MARC-Feldern zu holen.
#Es gibt unglaubliche viele Arten von MARC-Feldern (mit und ohne Unterfelder, wiederholbar nicht wiederholbar etc.)
#Hier wurden für zwei Arten Funktionen erstellt:
#1. Unwiederholbare Felder ohne Unterfeld (z.B. 001 mit MMS-ID)
#2. Felder, die mehrmals vorkommen können. Die Unterfelder sind aber pro Feld einmalig. (z.B. Schlagwörter in 6XX-Felder)
#Mit diesem Script sind also nicht alle Arten von Feldern abgedeckt. Mit einer weiteren Funktion kann dies aber hier ergänzt werden.


def unwiederholbare_ohne_unterfeld(feld_tag):
    #Bei beiden Feldern wurde try- und except eingefügt, weil ansonsten häufig aufgrund eines fehlerhaften Datensatzes das ganze Script abgebrochen wird. Aber daher ist auch empfehlenswert zu prüfen, ob wirklich alle Datensätze in mein CSV übernommen wurden.
    try:
        unwiederholbares_feld = soup_alles.find(tag=feld_tag).get_text()
        return(unwiederholbares_feld)
    except:
        pass
    
def unterfeldurchgehen(feld_tag, unterfeld_code):
    try:
        feldname_list = [unterfeld.find(code=unterfeld_code).get_text() 
                         for unterfeld in soup_alles.find_all(tag=feld_tag)]
        #wenn es mehere MARC-Feldern mit den gleichen Unterfeldern gibt, werden die Unterfelder mit einem | abgetrennt
        #weil es bei der Verarbeitung von csv in Excel immer Probleme gibt, wurden hier ; zu / umgewandelt. Dies ist aber fakultativ.
        beautiful_string = "| ".join(feldname_list).replace(";", "/")
        return beautiful_string
    except:
        pass


#Hier beginnt die eigentliche Programmschleife. Die einzelnen URLs mit dem Zusatz werden laufend zum SRU-String zusammengefügt.
  
for cur_parameter in liste_paramater_links:
    sru_strings= sru_basis + str(cur_parameter)
 
    sru_data_alles = urllib.request.urlopen(sru_strings).read()
   
 #Damit die SRU-Schnittselle nicht überfodert wird, werden die Abfragen nur im 2-Sekunden-Takt ausgeführt.   
    time.sleep(2)
#Mit Hilfe der Bibliothek "BeautifulSoup" werden die erhaltenen Daten als XML erkannt und können ausgelesen werden.
    soup_alles = BeautifulSoup(sru_data_alles, 'xml')
    
   
 #Die letzte Zeile kann nach Bedarf angepasst werden. Hier werden nämlich die MARC-Felder definiert, die in der CSV-Datei auglesen werden sollen.
#Dafür werden die oben definierten Funktionen ausgeführt. Wichtig ist, dass die ausgewählten MARC-Felder mit den oben definierten Spaltennamen übereinstimmten.
    f.writerow([ unwiederholbare_ohne_unterfeld("001"), unterfeldurchgehen("110", "a"), unterfeldurchgehen("110", "0"), unterfeldurchgehen("710", "a"), unterfeldurchgehen("710", "0"), unterfeldurchgehen("520", "a"), unterfeldurchgehen("245", "a"),  unterfeldurchgehen("245", "p"), unterfeldurchgehen("856", "u"), unterfeldurchgehen("990", "f"),  unwiederholbare_ohne_unterfeld("008") , unterfeldurchgehen("264", "c") ])



### Beispiel um die erhaltenen CSV-Dateien miteinander abzugleichen

Dabei ist die Bibliothek "Pandas" und deren Befehl "merge" sehr nützlich.

In [30]:
#Bibliotheken importieren
import pandas as pd
import csv

#CSV-Dateien öffnen und eine Variable als Titel vergeben. 
#Achtung: Die CSV-Dateien müssen im gleichen Ordner wie das Jupyter-Notebook oder das py-file gespeichert sein.
dizas = pd.read_csv("daten_dump_doksdizs.csv",sep=",")
ezas = pd.read_csv("daten_dump_doksezs.csv",sep=",")

#mergen von zwei verschiedenen Tabellen ahnad von Spaltennamen (im Beispiel: 110 0). 
#Der Befehlt pd.notnull macht, dass leere Spalten nicht "gemergt" werden.

ezas.merge(dizas[pd.notnull(dizas['110 0'])], on='110 0')

#gemergete Tabelle wieder als csv-abspeichern

gemergtes_csv = ezas.merge(dizas[pd.notnull(dizas['110 0'])], on='110 0')

gemergtes_csv.to_csv( "combined_csv.csv", encoding='utf-16')